In [1]:
import os
import numpy as np

# if not os.path.exists("chip1.bin"):
  # !wget https://users.fit.cvut.cz/bucekj/SRAM_data.zip
  # !unzip SRAM_data.zip

In [2]:
# Load PUF responses from files
# file contents is 1000 x 512B, i.e. 1000 x 512 x 8 b

MEASUREMENTS = 1000
MEM_SIZE = 512*8
CHIPS = 10

memories = np.array([
    np.unpackbits(
        np.fromfile(f"chip{i+1}.bin", dtype='uint8')
    ).reshape(MEASUREMENTS, MEM_SIZE)
    for i in range(CHIPS)
])

In [3]:
memories.shape

(10, 1000, 4096)

In [4]:
# USEFUL HINT
# Calculating mean (along specific axis)
# Try out different axes.
# What is the correct axis to produce a mean response of each chip?
print(memories.mean())
print(memories.mean(axis=1))
print(memories.mean(axis=1).shape)

0.491569384765625
[[0.    0.    1.    ... 0.787 1.    0.   ]
 [0.    1.    0.473 ... 0.029 1.    0.   ]
 [1.    0.    1.    ... 0.    0.107 1.   ]
 ...
 [0.756 0.    1.    ... 0.    1.    0.   ]
 [1.    1.    0.996 ... 0.963 1.    0.   ]
 [1.    0.755 0.    ... 1.    1.    0.   ]]
(10, 4096)


In [6]:
# USEFUL HINT
# Find elements satisfying specific condition => conversion to array of boolean values
a=np.array([5,4,1,1,8,9,1,10,0,11])
b=np.array([1,4,99,54,11,67,21,0,18,84])
a2 = a>4
b2 = b<70
print(a2)
print(b2)
a2^b2 # XOR of two boolean arrays

[ True False False False  True  True False  True False  True]
[ True  True False  True  True  True  True  True  True False]


array([False,  True, False,  True, False, False,  True, False,  True,
        True])

In [7]:
# Create reference responses
avgs = memories.mean(axis=1)

refs = avgs > 0.5 # ??? # reference responses (array of boolean values?)
                     
# refs = ??? # reference responses (array of boolean values?)

print(refs.shape)
print(refs)

(10, 4096)
[[False False  True ...  True  True False]
 [False  True False ... False  True False]
 [ True False  True ... False False  True]
 ...
 [ True False  True ... False  True False]
 [ True  True  True ...  True  True False]
 [ True  True False ...  True  True False]]


In [8]:
# To compute HDintra, we need to be able to calculate hamming distance between responses and reference responses
# Hamming distance can be calculated as: HD(x, y) = HW(x xor y)
# Example:
x = np.array([1,0,1,0])
y = np.array([1,1,0,0])
# print(x ^ y)
print(x)
print(x[1:])

[1 0 1 0]
[0 1 0]


In [9]:
# Responses and reference responses have different dimensions
print(refs.shape)
print(memories.shape)

# HINT
# Add missing dimensions by indexing with None (or np.newaxis)
# Use the Numpy broadcasting mechanism to stretch one array to the shape of the other
# https://numpy.org/doc/stable/user/basics.broadcasting.html
x = np.array([[1,0,1,0], [1,1,1,1], [0,0,0,0]])
y = np.array([[1,1,0,0], [0,0,1,1]])
print(x.shape)
print(y.shape)
print("added")
print(y[:,None].shape)
z=y[:, None]^x
print(z)
print(z.shape)

(10, 4096)
(10, 1000, 4096)
(3, 4)
(2, 4)
added
(2, 1, 4)
[[[0 1 1 0]
  [0 0 1 1]
  [1 1 0 0]]

 [[1 0 0 1]
  [1 1 0 0]
  [0 0 1 1]]]
(2, 3, 4)


In [10]:
# Compute HDintra
xors = refs[:,None, :] ^ memories
print(xors.shape)
print(xors)

# ??? # finish HDintra calculation (look at the definition)
hamming_distances = np.sum(xors, axis=2)
# value_for_mask = np.sum(xors, axis=1)
# print(value_for_mask.shape)
# print(value_for_mask)
print(hamming_distances.shape)
total_hamming_distances = np.sum(hamming_distances, axis=-1)
print(total_hamming_distances.shape)
HD_intra = total_hamming_distances.mean() / (1000*4096) * 100
print(HD_intra)


# MEASUREMENTS = 1000
# MEM_SIZE = 512*8
# CHIPS = 10

(10, 1000, 4096)
[[[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 1 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 1 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 1 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 1 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 ...

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]

 [[0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 1 0 ... 0 0 0]
  ...
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]
  [0 0 0 ... 0 0 0]]]
(10, 1000)
(10,)
3.0485107421875


In [11]:
# Compute HDinter
# We need to compute HD of all pairs of reference responses
idxs = np.triu_indices(CHIPS, 1)
# returns a tuple of indexes of upper triangle of N x N matrix (N=CHIPS in this case)
# - useful for all possible pairwise combinations of elements in one array (reference responses in our case)
print(idxs)
print(refs[idxs[0]].shape)
print(refs[idxs,:].shape)
pairwise_xors = refs[idxs[0]] ^ refs[idxs[1]]
hamming_distances = pairwise_xors.sum(axis=-1)
total_hamming_distances = hamming_distances.sum(axis=-1)

HD_inter = total_hamming_distances * 2 / (CHIPS*(CHIPS-1)) * 100 /4096
print(HD_inter)

(array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2,
       2, 2, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7,
       8]), array([1, 2, 3, 4, 5, 6, 7, 8, 9, 2, 3, 4, 5, 6, 7, 8, 9, 3, 4, 5, 6, 7,
       8, 9, 4, 5, 6, 7, 8, 9, 5, 6, 7, 8, 9, 6, 7, 8, 9, 7, 8, 9, 8, 9,
       9]))
(45, 4096)
(2, 45, 4096)
43.57313368055556


In [25]:
# Create masks
e = 0.1 # Choose an acceptable error rate to be included into the mask

value_for_mask = np.sum(xors, axis=1)
print(value_for_mask.shape)
value_for_mask = value_for_mask/1000
print(value_for_mask)
# print(value_for_mask)
masks = (value_for_mask > (1-e)) | (value_for_mask < e)
masks.shape
print(masks)
print(np.sum(masks[0]))

(10, 4096)
[[0.    0.    0.    ... 0.213 0.    0.   ]
 [0.    0.    0.473 ... 0.029 0.    0.   ]
 [0.    0.    0.    ... 0.    0.107 0.   ]
 ...
 [0.244 0.    0.    ... 0.    0.    0.   ]
 [0.    0.    0.004 ... 0.037 0.    0.   ]
 [0.    0.245 0.    ... 0.    0.    0.   ]]
[[ True  True  True ... False  True  True]
 [ True  True False ...  True  True  True]
 [ True  True  True ...  True False  True]
 ...
 [False  True  True ...  True  True  True]
 [ True  True  True ...  True  True  True]
 [ True False  True ...  True  True  True]]
3649


In [19]:
# Save masks into file
np.savetxt('masks.txt', masks.astype(int), fmt='%d', delimiter='')

In [20]:
# Create masked responses (1024 bits long)
mask_idx = np.array([np.nonzero(masks[i])[0][:1024] for i in range(10)])
memories_masked = np.array([memories[i][:,mask_idx[i]] for i in range(10)])
refs_masked = np.array([refs[i][mask_idx[i]] for i in range(10)])
print(refs_masked)
print(refs_masked.shape)

print(memories_masked.shape)

[[False False  True ...  True False  True]
 [False  True False ... False  True False]
 [ True False  True ... False  True False]
 ...
 [False  True False ... False False  True]
 [ True  True  True ... False  True  True]
 [ True False False ... False  True False]]
(10, 1024)
(10, 1000, 1024)


In [26]:
# Compute HDintra of masked PUF responses
# Compute HDintra
xors_2 = refs_masked[:,None, :] ^ memories_masked
# print(xors.shape)
print(xors_2.shape)

# ??? # finish HDintra calculation (look at the definition)
hamming_distances = np.sum(xors_2, axis=1)
hamming_distances = hamming_distances/1000
# print(hamming_distances.shape)
print(hamming_distances.shape)
total_hamming_distances = np.sum(hamming_distances, axis=-1)
total_hamming_distances = total_hamming_distances/1024
print(total_hamming_distances.shape)
print(total_hamming_distances)
masked_HD_intra = total_hamming_distances.mean()*100
print(masked_HD_intra)

(10, 1000, 1024)
(10, 1024)
(10,)
[0.00906934 0.01079492 0.00967578 0.00985547 0.00836035 0.00753809
 0.00950977 0.00818457 0.00956641 0.00788867]
0.90443359375


In [27]:
# Compute HDinter of masked PUF responses
# Compute HDinter
# We need to compute HD of all pairs of reference responses
idxs = np.triu_indices(CHIPS, 1)
# returns a tuple of indexes of upper triangle of N x N matrix (N=CHIPS in this case)
# - useful for all possible pairwise combinations of elements in one array (reference responses in our case)
# print(idxs)
# print(refs[idxs[0]].shape)
# print(refs[idxs,:].shape)
pairwise_xors = refs_masked[idxs[0]] ^ refs_masked[idxs[1]]
hamming_distances = pairwise_xors.sum(axis=-1)
total_hamming_distances = hamming_distances.sum(axis=-1)

masked_HD_inter = total_hamming_distances * 2 / (CHIPS*(CHIPS-1)) * 100 /1024
print(masked_HD_inter)

50.14756944444444


In [28]:
# Print a summary of results (HDintra, HDinter - for both unmasked and masked PUF responses)
print(HD_intra, HD_inter, masked_HD_intra, masked_HD_inter)

3.0485107421875 43.57313368055556 0.90443359375 50.14756944444444
